# What do Principal Components Actually do Mathematically?
I have recently taken an interest in PCA after watching Professor Gilbert Strang’s [PCA lecture](https://www.youtube.com/watch?v=Y4f7K9XF04k). I must have watched at least 15 other videos and read 7 different blog posts on PCA since. They are all very excellent resources, but I found myself somewhat unsatisfied. What they do a lot is teaching us the following:
- What the PCA promise is;
- Why that promise is very useful in Data Science; and
- How to extract these principal components. (Although I don't agree with how some of them do it by applying SVD on the covariance matrix, that can be saved for another post.)

Some of them go the extra mile to show us graphically or logically, how the promise is being fulfilled by the principal components:
- Graphically, a transformed vector can be shown to be still clustered with its original group in a plot; and
- Logically, a proof can be expressed in mathematical symbols.

## Objective
My mind was convinced, but my heart was still not. To me, the graphic approach does not provide a visual effect that is striking enough. The logic approach, on the other hand, does not even spend enough effort to explain what precisely it's trying to prove. Therefore, the objective of this post is to improve both of these 2 areas - to provide a more striking visual and to establish a more precise goal for our mathematical proof.

## Prerequisites
This post is for you if:
- You have at least a slight idea of what the PCA promise is;
- You have a decent understanding of what the covariance matrix is about;
- You have a solid foundation in linear algebra, e.g., comfortable with the concept of matrices being transformations that rotate and/or stretch spaces; and
- Your heart is longing to discover the principal components, instead of being told what they are!

## How to Choose P?
After hearing my dissatisfaction, my friend [Calvin](https://calvinfeng.github.io/) recommended this paper by Jonathon Shlens - [A Tutorial on Principal Component Analysis](https://arxiv.org/pdf/1404.1100.pdf) to me. It is by far the best resource I have come across on PCA. However, it's also a bit lengthier than your typical blog post, so the remainder of this post will focus on section 5 of the paper. In there, Jonathon immediately establishes the following goal:
> The [original] dataset is $X$, an $m × n$ matrix.<br>
> Find some orthonormal matrix $P$ in $Y = PX$ such that $C_Y \equiv \frac{1}{n}YY^T$ is a diagonal matrix.[1]<br>
> The rows of $P$ shall be principal components of $X$.

As you might have noticed, $C_Y$ here is the covariance matrix of our rotated dataset $Y$. Why do we want $C_Y$ to be diagonal? Before we answer this question, let’s generate a dataset $X$ consisting of 4 features with some random values.

In [202]:
from IPython.display import Latex, display
from string import ascii_lowercase
import numpy as np
import pandas as pd

# constants
FEAT_NUM, SAMPLE_NUM = 4, 4

# helpers
def covariance_matrix(dataset):
    return dataset @ dataset.transpose() / SAMPLE_NUM

def tabulate(dataset, rotated=False):
    '''
    Label row(s) and column(s) of a matrix by wrapping it in a dataframe.
    '''
    if rotated:
        prefix = 'new_'
        feats = ascii_lowercase[FEAT_NUM:2 * FEAT_NUM]
    else:
        prefix = ''
        feats = ascii_lowercase[0:FEAT_NUM]
    return pd.DataFrame.from_records(dataset, 
                                     columns=['sample{}'.format(num) for num in range(SAMPLE_NUM)], 
                                     index=['{}feat_{}'.format(prefix, feat) for feat in feats])

def display_df(dataset, latex=False):
    rounded = dataset.round(15)
    if latex:
        display(Latex(rounded.to_latex()))
    else:
        display(rounded)

In [206]:
x = tabulate(np.random.rand(FEAT_NUM, SAMPLE_NUM))
display_df(x)

,sample0,sample1,sample2,sample3
feat_a,0.960266,0.541859,0.263176,0.922409
feat_b,0.874243,0.355070,0.309325,0.785072
feat_c,0.484207,0.594282,0.566241,0.677418
feat_d,0.600766,0.775064,0.861898,0.299897


Looks just like any other normal dataset. Nothing special. What about its covariance matrix?

In [207]:
c_x = covariance_matrix(x)
display_df(c_x)

,feat_a,feat_b,feat_c,feat_d
feat_a,0.533955,0.459367,0.390216,0.375082
feat_b,0.459367,0.400599,0.335325,0.325616
feat_c,0.390216,0.335325,0.341788,0.360675
feat_d,0.375082,0.325616,0.360675,0.448613


![alt text](doesnt_look_like_anything.jpg "Doesn't look like anything to me")

In [203]:
x = tabulate(np.random.rand(FEAT_NUM, SAMPLE_NUM))
c_x = covariance_matrix(x)
_, p = np.linalg.eig(c_x)
y = tabulate(p.transpose() @ x, rotated=True)
c_y = covariance_matrix(y)

In [204]:
display_df(x)
display_df(c_x)
display_df(y)
display_df(c_y)

,sample0,sample1,sample2,sample3
feat_a,0.032694,0.680192,0.675898,0.585956
feat_b,0.229547,0.628407,0.402396,0.218725
feat_c,0.951576,0.404422,0.878310,0.917486
feat_d,0.814071,0.470929,0.099848,0.923725


,feat_a,feat_b,feat_c,feat_d
feat_a,0.315979,0.208771,0.359363,0.238922
feat_b,0.208771,0.164338,0.256670,0.181256
feat_c,0.359363,0.256670,0.670566,0.475077
feat_d,0.238922,0.181256,0.475077,0.436931


,sample0,sample1,sample2,sample3
new_feat_e,1.156732,0.991074,1.054079,1.416422
new_feat_f,-0.506006,0.401910,0.422045,-0.182063
new_feat_g,-0.147386,-0.102179,0.002545,0.189964
new_feat_h,-0.078583,0.297806,-0.333089,0.103679


,new_feat_e,new_feat_f,new_feat_g,new_feat_h
new_feat_e,1.359398,-0.000000,0.000000,-0.00000
new_feat_f,-0.000000,0.157211,-0.000000,-0.00000
new_feat_g,0.000000,-0.000000,0.017064,0.00000
new_feat_h,-0.000000,-0.000000,0.000000,0.05414


In [205]:
display_df(x, latex=True)
display_df(c_x, latex=True)
display_df(y, latex=True)
display_df(c_y, latex=True)

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

[1]: The reason orthonormality is part of the goal is that we do not want to do anything more than rotating $X$. We do not want to modify $X$. We only want to re-express $X$ by carefully choosing a change of basis.